### reproduce the Flaxman et al analysis

https://github.com/ImperialCollegeLondon/covid19model/releases

Estimating the number of infections and the impact of non-pharmaceutical interventions on COVID-19 in 11 European countries

https://www.imperial.ac.uk/media/imperial-college/medicine/sph/ide/gida-fellowships/Imperial-College-COVID19-Europe-estimates-and-NPI-impact-30-03-2020.pdf

Try to reproduce the `base.r` code -- sets up the stan model, 

In [1]:
import os
import datetime

import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

import pystan

from statsmodels.distributions.empirical_distribution import ECDF

from pystanutils import pystan_utils   ### local imports
from pystanutils import stan_utility

### Some utility functions reproducing R functions

In [2]:
from scipy.stats import gamma

def poly(x, p):
    """ 
    equivalent of R `poly function. See 
    https://stackoverflow.com/questions/41317127/python-equivalent-to-r-poly-function
    """
    x = np.array(x)
    X = np.transpose(np.vstack(list(x**k for k in range(p+1))))
    return np.linalg.qr(X)[0][:,1:]

def decimal_date(date):
    """ convert to a decimalized year -- not really needed since datetime can compare and sort """
    return date.year + (date.dayofyear -1 )/(365+date.is_leap_year)

def pgammaAlt(q, mean, cv=1):
    """
    alternative parameterization of gamma cdf
    see http://search.r-project.org/library/EnvStats/html/GammaAlt.html
    """
    shape = 1/cv**2
    scale = mean/shape
    return gamma.cdf(q, shape, scale=scale)

def rgammaAlt(n, mean, cv=1):
    """
    alternative parameterization of gamma random numbers
    see http://search.r-project.org/library/EnvStats/html/GammaAlt.html
    """
    shape = 1/cv**2
    scale = mean/shape
    return gamma.rvs(shape, scale=scale, size=n)



In [3]:
countries = [  "Denmark",
  "Italy",
  "Germany",
  "Spain",
  "United_Kingdom",
  "France",
  "Norway",
  "Belgium",
  "Austria", 
  "Sweden",
  "Switzerland"]

#countries = ['United_Kingdom',]

ddir = "../data"
StanModel = 'base'

In [4]:
#ecdc_data_csv = 'https://opendata.ecdc.europa.eu/covid19/casedistribution/csv'  ### get latest
ecdc_data_csv = f"{ddir}/COVID-19-up-to-date.csv"    ### file from repo
d = pd.read_csv(ecdc_data_csv, parse_dates=['dateRep'], dayfirst=True, encoding='ISO-8859-1')

In [5]:
## convert to decimal date, although not actually needed?
d['t'] = decimal_date(d.dateRep.dt)

d=d.rename(columns={
    "countriesAndTerritories": "Countries_and_territories", 
    "deaths": "Deaths", 
    "cases": "Cases",
    "dateRep": "DateRep"});

In [6]:
ifr_by_country = pd.read_csv(f"{ddir}/weighted_fatality.csv", )
ifr_by_country['country'] = ifr_by_country.iloc[:,1]
ifr_by_country.loc[ifr_by_country['country']=='United Kingdom', 'country'] = 'United_Kingdom'

In [7]:
serial_interval = pd.read_csv(f"{ddir}/serial_interval.csv")
covariates = pd.read_csv(f"{ddir}/interventions.csv", parse_dates=[1,2,3,4,5,6,7], dayfirst=True, nrows=11)
covariates = covariates.iloc[:11, :8]

In [8]:
## need to check this -- gives multiple SettingWithCopyWarning w/o *.loc
covariates.schools_universities.loc[covariates.schools_universities > covariates.lockdown] = covariates.lockdown.loc[covariates.schools_universities > covariates.lockdown]
covariates.travel_restrictions.loc[covariates.travel_restrictions > covariates.lockdown] = covariates.lockdown.loc[covariates.travel_restrictions > covariates.lockdown] 
covariates.public_events.loc[covariates.public_events > covariates.lockdown] = covariates.lockdown.loc[covariates.public_events > covariates.lockdown]
covariates.sport.loc[covariates.sport > covariates.lockdown] = covariates.lockdown.loc[covariates.sport > covariates.lockdown]
covariates.social_distancing_encouraged.loc[covariates.social_distancing_encouraged > covariates.lockdown] = covariates.lockdown.loc[covariates.social_distancing_encouraged > covariates.lockdown]
covariates.self_isolating_if_ill.loc[covariates.self_isolating_if_ill > covariates.lockdown] = covariates.lockdown.loc[covariates.self_isolating_if_ill > covariates.lockdown]


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [9]:
p = covariates.shape[1]-1   # number of columns
forecast = 0

In [10]:
DEBUG = False
if not DEBUG:
  N2 = 75 # Increase this for a further forecast
else:
  ### For faster runs:
  # countries = c("Austria","Belgium") #,Spain")
  N2 = 75

# countries = c("Italy","United_Kingdom","Spain","Norway","Austria","Switzerland")

dates = {}
reported_cases = {}
stan_data = {'M': len(countries), 
             'N': [],
             'p': p,
             'x1': poly(range(N2),2)[:,0],
             'x2': poly(range(N2),2)[:,1],
             'y': [],
             'covariate1': [],
             'covariate2': [],
             'covariate3': [],
             'covariate4': [],
             'covariate5': [],
             'covariate6': [],
             'covariate7': [],
             'deaths': [],
             'f': [],
             'N0': 6,
             'cases': [],
             'LENGTHSCALE': 7,
             'SI': serial_interval.fit[:N2],
             'EpidemicStart': []} # N0 = 6 to make it consistent with Rayleigh
deaths_by_country = {}


In [11]:
for Country in countries:

    IFR=ifr_by_country.weighted_fatality[ifr_by_country.country == Country].values[0]

    covariates1 = covariates[covariates.Country == Country].iloc[:,1:]

    d1 = d[d.Countries_and_territories==Country]
    d1 = d1.sort_values('t')
    index = np.where(d1.Cases>0)[0][0]
    index1 = np.where(np.cumsum(d1.Deaths)>=10)[0][0] # also 5
    index2 = index1-30
    print(f"First non-zero cases is on day {index}, and 30 days before 5 days is day {index2}")
    d1=d1.iloc[index2:]
    stan_data['EpidemicStart'].append(index1+1-index2) 

    for cov in covariates1.columns:
            d1[cov] =  (d1.DateRep >= covariates1.loc[:,cov].values[0])  # should this be > or >=?
            ### AHJ: need to coerce to just a value not a series for the comparison
            
            
    dates[Country] = d1.DateRep
    # hazard estimation
    N = len(d1.Cases)
    print(f"{Country} has {N} days of data")
    forecast = N2 - N
    if forecast < 0:
        print(f"{Country}: {N}")
        print("ERROR!!!! increasing N2")
        N2 = N
        forecast = N2 - N

    print("N, N2, forecast = ", N, N2, forecast)
    h = []
    DEBUG = False
    if DEBUG:  # OLD -- but faster for testing this part of the code
        mean = 18.8
        cv = 0.45

        for i in range(1, forecast+N+1):
            h1 = (IFR*pgammaAlt(i, mean, cv=cv) - IFR*pgammaAlt(i-1, mean, cv=cv))/(1-IFR*pgammaAlt(i-1, mean, cv=cv))
            h.append(h1)
    else:
        mean1 = 5.1; cv1 = 0.86; # infection to onset
        mean2 = 18.8; cv2 = 0.45 # onset to death
        ## assume that IFR is probability of dying given infection
        x1 = rgammaAlt(int(5e6), mean1, cv1) # infection-to-onset ----> do all people who are infected get to onset?
        x2 = rgammaAlt(int(5e6), mean2, cv2) # onset-to-death
        fc = ECDF(x1+x2)
        convolution = lambda u: IFR * fc(u)

        h.append(convolution(1.5) - convolution(0)) 
        for i in range(2, forecast+N+1):
            h.append((convolution(i+.5) - convolution(i-.5)) / (1-convolution(i-.5)))
        h = np.array(h)
            
    s = [1,] 
    for i in range(1,N2):
        s.append(s[i-1]*(1-h[i-1]))
    s = np.array(s)

    f = s * h

    y = np.hstack((d1.Cases.values,-np.ones(forecast, dtype=np.int)))
    reported_cases[Country] = d1.Cases
    deaths = np.hstack((d1.Deaths.values, -np.ones(forecast, dtype=np.int)))
    cases = np.hstack((d1.Cases.values, -np.ones(forecast, dtype=np.int)))
    deaths_by_country[Country] = d1.Deaths
    covariates2 = d1[covariates1.columns]
    # x=1:(N+forecast)    
    ### append copies of the last row so that it has length N+forecast
    covariates2 = covariates2.append(pd.DataFrame(covariates2.iloc[-1:].values.repeat(forecast, axis=0), 
                                                  columns=covariates2.columns, index=np.arange(N, N+forecast)))
    #     ## append data
    stan_data['N'].append(N)
    stan_data['y'].append(y[0]) # just the index case!
    # stan_data.x = cbind(stan_data.x,x)
    stan_data['covariate1'].append(covariates2.iloc[:,0].values)
    stan_data['covariate2'].append(covariates2.iloc[:,1].values)
    stan_data['covariate3'].append(covariates2.iloc[:,2].values)
    stan_data['covariate4'].append(covariates2.iloc[:,3].values) 
    stan_data['covariate5'].append(covariates2.iloc[:,4].values)
    stan_data['covariate6'].append(covariates2.iloc[:,5].values)
    stan_data['covariate7'].append(covariates2.iloc[:,6].values)
    stan_data['f'].append(f)
    stan_data['deaths'].append(deaths)
    stan_data['cases'].append(cases)

    stan_data['N2'] = N2
    stan_data['x'] = np.arange(1,N2+1)
    

First non-zero cases is on day 58, and 30 days before 5 days is day 52
Denmark has 37 days of data
N, N2, forecast =  37 75 38
First non-zero cases is on day 31, and 30 days before 5 days is day 27
Italy has 62 days of data
N, N2, forecast =  62 75 13
First non-zero cases is on day 28, and 30 days before 5 days is day 46
Germany has 43 days of data
N, N2, forecast =  43 75 32
First non-zero cases is on day 32, and 30 days before 5 days is day 40
Spain has 49 days of data
N, N2, forecast =  49 75 26
First non-zero cases is on day 31, and 30 days before 5 days is day 43
United_Kingdom has 46 days of data
N, N2, forecast =  46 75 29
First non-zero cases is on day 25, and 30 days before 5 days is day 38
France has 51 days of data
N, N2, forecast =  51 75 24
First non-zero cases is on day 58, and 30 days before 5 days is day 55
Norway has 34 days of data
N, N2, forecast =  34 75 41
First non-zero cases is on day 35, and 30 days before 5 days is day 49
Belgium has 40 days of data
N, N2, fore

In [12]:
for i in range(7):
    stan_data[f'covariate{i+1}'] = np.array(stan_data[f'covariate{i+1}'])

stan_data['covariate2'] = 0 # remove travel bans   ## doesn't do anything but useful reminder
stan_data['covariate4'] = 0 # remove sport

#stan_data$covariate1 = stan_data$covariate1 # school closure
stan_data['covariate2'] = stan_data['covariate7'] # self-isolating if ill
#stan_data$covariate3 = stan_data$covariate3 # public events
# create the `any intervention` covariate
stan_data['covariate4'] = (stan_data['covariate1']+
                           stan_data['covariate3']+
                           stan_data['covariate5']+
                           stan_data['covariate6']+
                           stan_data['covariate7'])
# stan_data$covariate5 = stan_data$covariate5 # lockdown
# stan_data$covariate6 = stan_data$covariate6 # social distancing encouraged
stan_data['covariate7'] = 0 # models should only take 6 covariates

for i in range(7):
    stan_data[f'covariate{i+1}'] = np.int_(stan_data[f'covariate{i+1}']).transpose()
    
for label in ('cases', 'deaths', 'f'):
    stan_data[label] = np.array(stan_data[label]).transpose()

In [13]:
dbg = True
if dbg:
    resdir = './results'
    if not os.path.exists(resdir):
        os.makedirs(resdir)
    for i, co in enumerate(countries):
        nr = slice(0,stan_data['N'][i])
        with open(f"{resdir}/{co}-check-dates-python.csv", 'w') as f:
            pd.DataFrame({
                'date': dates[co],
                'school closure': stan_data['covariate1'][nr,i],
                'self isolating if ill': stan_data['covariate2'][nr,i],
                'public event': stan_data['covariate3'][nr,i],
                'government makes any intervention': stan_data['covariate4'][nr,i],
                'lockdown': stan_data['covariate5'][nr,i],
                'social distancing encouraged': stan_data['covariate6'][nr,i]
              }).to_csv(f)

            

In [14]:
stan_data

{'M': 11,
 'N': [37, 62, 43, 49, 46, 51, 34, 40, 36, 40, 44],
 'p': 7,
 'x1': array([-1.97350876e-01, -1.92017069e-01, -1.86683261e-01, -1.81349454e-01,
        -1.76015647e-01, -1.70681839e-01, -1.65348032e-01, -1.60014224e-01,
        -1.54680417e-01, -1.49346609e-01, -1.44012802e-01, -1.38678994e-01,
        -1.33345187e-01, -1.28011379e-01, -1.22677572e-01, -1.17343764e-01,
        -1.12009957e-01, -1.06676149e-01, -1.01342342e-01, -9.60085345e-02,
        -9.06747270e-02, -8.53409195e-02, -8.00071121e-02, -7.46733046e-02,
        -6.93394971e-02, -6.40056896e-02, -5.86718822e-02, -5.33380747e-02,
        -4.80042672e-02, -4.26704598e-02, -3.73366523e-02, -3.20028448e-02,
        -2.66690374e-02, -2.13352299e-02, -1.60014224e-02, -1.06676149e-02,
        -5.33380747e-03,  2.64323941e-18,  5.33380747e-03,  1.06676149e-02,
         1.60014224e-02,  2.13352299e-02,  2.66690374e-02,  3.20028448e-02,
         3.73366523e-02,  4.26704598e-02,  4.80042672e-02,  5.33380747e-02,
         5.

In [15]:
stanc_ret = pystan.stanc(file=f'../stan-models/{StanModel}.stan', model_name=f'Cov19_{StanModel}')
sm = stan_utility.StanModel_cache(stanc_ret=stanc_ret)

Using cached StanModel


In [19]:
fit = sm.sampling(data=stan_data, iter=2000, warmup=1000, chains=4, thin=4, 
                  control={'adapt_delta': 0.90, 'max_treedepth': 10})

To run all diagnostics call pystan.check_hmc_diagnostics(fit)


In [20]:
stan_utility.check_all_diagnostics(fit)

n_eff / iter looks reasonable for all parameters
Rhat for parameter lp0[38,1] is nan!
Rhat for parameter lp0[39,1] is nan!
Rhat for parameter lp0[40,1] is nan!
Rhat for parameter lp0[41,1] is nan!
Rhat for parameter lp0[42,1] is nan!
Rhat for parameter lp0[43,1] is nan!
Rhat for parameter lp0[44,1] is nan!
Rhat for parameter lp0[45,1] is nan!
Rhat for parameter lp0[46,1] is nan!
Rhat for parameter lp0[47,1] is nan!
Rhat for parameter lp0[48,1] is nan!
Rhat for parameter lp0[49,1] is nan!
Rhat for parameter lp0[50,1] is nan!
Rhat for parameter lp0[51,1] is nan!
Rhat for parameter lp0[52,1] is nan!
Rhat for parameter lp0[53,1] is nan!
Rhat for parameter lp0[54,1] is nan!
Rhat for parameter lp0[55,1] is nan!
Rhat for parameter lp0[56,1] is nan!
Rhat for parameter lp0[57,1] is nan!
Rhat for parameter lp0[58,1] is nan!
Rhat for parameter lp0[59,1] is nan!
Rhat for parameter lp0[60,1] is nan!
Rhat for parameter lp0[61,1] is nan!
Rhat for parameter lp0[62,1] is nan!
Rhat for parameter lp0[63,

In [21]:
pystan.check_hmc_diagnostics(fit)

{'n_eff': False,
 'Rhat': False,
 'divergence': True,
 'treedepth': True,
 'energy': True}

In [24]:
fit.summary()

OrderedDict([('summary',
              array([[3.39634112e+00, 2.64865792e-02, 7.87302273e-01, ...,
                      5.26469695e+00, 8.83550279e+02, 1.00109750e+00],
                     [3.58200452e+00, 8.66969140e-03, 2.82666979e-01, ...,
                      4.18094803e+00, 1.06302353e+03, 1.00555400e+00],
                     [4.35768908e+00, 2.61420569e-02, 7.47156826e-01, ...,
                      6.12177579e+00, 8.16853218e+02, 1.00149111e+00],
                     ...,
                     [2.44980109e+05, 2.96828004e+04, 9.41247057e+05, ...,
                      1.55871348e+06, 1.00553576e+03, 1.00246688e+00],
                     [3.31502818e+05, 4.25536983e+04, 1.35065523e+06, ...,
                      2.13116329e+06, 1.00742887e+03, 1.00245134e+00],
                     [8.45441856e+04, 1.44092773e-01, 4.20159717e+00, ...,
                      8.45514344e+04, 8.50245661e+02, 9.99928283e-01]])),
             ('c_summary',
              array([[[3.48408949e+00, 3.42

In [26]:
print(fit.stansummary())

Inference for Stan model: Cov19_base_cac77d4c122b5d474c43fdbfc9c71646.
4 chains, each with iter=2000; warmup=1000; thin=4; 
post-warmup draws per chain=250, total post-warmup draws=1000.

                      mean se_mean      sd     2.5%      25%      50%      75%     97.5%  n_eff   Rhat
mu[1]                  3.4    0.03    0.79     2.07     2.85      3.3      3.9      5.26    884    1.0
mu[2]                 3.58  8.7e-3    0.28     3.09     3.38     3.56     3.76      4.18   1063   1.01
mu[3]                 4.36    0.03    0.75     3.08     3.84     4.28      4.8      6.12    817    1.0
mu[4]                 4.73    0.02     0.5     3.83     4.39      4.7     5.05      5.86    595    1.0
mu[5]                 3.68    0.01    0.42     2.93     3.39     3.64     3.93       4.6    817    1.0
mu[6]                 3.89    0.01    0.35     3.24     3.64     3.88     4.11      4.62    887    1.0
mu[7]                 2.82    0.03    0.89     1.31     2.19     2.73     3.38      4.74   

In [ ]:
fit.